In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 617.7 kB/s eta 0:00:00m eta 0:00:010:00:09
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 889.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845499 sha256=3313a140c6cac7c68491535354b13f37d0fbe34535d71805278c46951edc3d45
  Stored in directory: /home/sharpnel/.cache/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [1]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("carCrash") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/24 23:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    


In [3]:
purchaseDataframe = spark.read.csv(
    "../resources/raw/Primary_Person_use.csv", 
    header=True, inferSchema=True, sep=",")
#show 3 rows of our DataFrame
print(purchaseDataframe.show(3))

23/01/24 23:36:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------+--------+--------+------------+------------------+-----------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+--------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|       PRSN_REST_ID|   PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_

In [4]:
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)

number of rows:  156954


In [5]:
male_killed_df = purchaseDataframe.filter("PRSN_GNDR_ID =='MALE' and  DEATH_CNT==1").select('CRASH_ID').distinct().count()

In [6]:
male_killed_df

180

In [7]:
purchaseDataframe.select('CRASH_ID','PRSN_GNDR_ID','DEATH_CNT').distinct().groupBy("PRSN_GNDR_ID","DEATH_CNT").count().alias("Death Count").where("DEATH_CNT==1").toPandas()

,PRSN_GNDR_ID,DEATH_CNT,count
0,MALE,1,180
1,FEMALE,1,64


In [8]:
purchaseDataframe.schema

StructType([StructField('CRASH_ID', IntegerType(), True), StructField('UNIT_NBR', IntegerType(), True), StructField('PRSN_NBR', IntegerType(), True), StructField('PRSN_TYPE_ID', StringType(), True), StructField('PRSN_OCCPNT_POS_ID', StringType(), True), StructField('PRSN_INJRY_SEV_ID', StringType(), True), StructField('PRSN_AGE', StringType(), True), StructField('PRSN_ETHNICITY_ID', StringType(), True), StructField('PRSN_GNDR_ID', StringType(), True), StructField('PRSN_EJCT_ID', StringType(), True), StructField('PRSN_REST_ID', StringType(), True), StructField('PRSN_AIRBAG_ID', StringType(), True), StructField('PRSN_HELMET_ID', StringType(), True), StructField('PRSN_SOL_FL', StringType(), True), StructField('PRSN_ALC_SPEC_TYPE_ID', StringType(), True), StructField('PRSN_ALC_RSLT_ID', StringType(), True), StructField('PRSN_BAC_TEST_RSLT', StringType(), True), StructField('PRSN_DRG_SPEC_TYPE_ID', StringType(), True), StructField('PRSN_DRG_RSLT_ID', StringType(), True), StructField('DRVR_D

In [15]:
from pyspark.sql.functions import col,isnan, when, count

In [16]:
purchaseDataframe.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in purchaseDataframe.columns]
   ).show()

AnalysisException: cannot resolve 'isnan(PRSN_DEATH_TIME)' due to data type mismatch: argument 1 requires (double or float) type, however, 'PRSN_DEATH_TIME' is of timestamp type.;
'Aggregate [count(CASE WHEN (isnan(cast(CRASH_ID#17 as double)) OR isnull(CRASH_ID#17)) THEN CRASH_ID END) AS CRASH_ID#403L, count(CASE WHEN (isnan(cast(UNIT_NBR#18 as double)) OR isnull(UNIT_NBR#18)) THEN UNIT_NBR END) AS UNIT_NBR#405L, count(CASE WHEN (isnan(cast(PRSN_NBR#19 as double)) OR isnull(PRSN_NBR#19)) THEN PRSN_NBR END) AS PRSN_NBR#407L, count(CASE WHEN (isnan(cast(PRSN_TYPE_ID#20 as double)) OR isnull(PRSN_TYPE_ID#20)) THEN PRSN_TYPE_ID END) AS PRSN_TYPE_ID#409L, count(CASE WHEN (isnan(cast(PRSN_OCCPNT_POS_ID#21 as double)) OR isnull(PRSN_OCCPNT_POS_ID#21)) THEN PRSN_OCCPNT_POS_ID END) AS PRSN_OCCPNT_POS_ID#411L, count(CASE WHEN (isnan(cast(PRSN_INJRY_SEV_ID#22 as double)) OR isnull(PRSN_INJRY_SEV_ID#22)) THEN PRSN_INJRY_SEV_ID END) AS PRSN_INJRY_SEV_ID#413L, count(CASE WHEN (isnan(cast(PRSN_AGE#2